In [14]:
import uiautomation as auto
from time import sleep
import pyautogui
from uiautomation import PatternId, ScrollAmount  # Patterns & scroll enums

In [10]:
# Suppose `node` is the UIA element you found earlier
sleep(4)

### click based on text

In [ ]:



node = auto.ButtonControl(Name='Read Aloud')  # or however you identified it
if node.Exists(0, 0):
    node.Click()  # Invokes the control directly via UIA
print("done")

done


In [8]:
screen_width, screen_height = pyautogui.size()

print(f"Max coordinates: ({screen_width}, {screen_height})")

Max coordinates: (1920, 1080)


### click based on coord

In [4]:
sleep(4)

# Define the coordinates from your output (middle of the element)
x = (608 + 863) // 2  # middle x coordinate
y = (1594 + 1618) // 2  # middle y coordinate

# Move mouse and click at those coordinates
auto.Click(x, y)
print(f"Clicked at coordinates: ({x}, {y})")

Clicked at coordinates: (735, 1606)


### click based on the output file

In [12]:
sleep(2)
N = 1   # change to 2 for the 2nd match, etc.
ctrl = auto.TextControl(
    Name='✅ Started 2 startups (both fake)',
    FoundIndex=N
)

if not ctrl.Exists(0, 0):
    print(f"No matching element #{N} found.")
else:
    # 3) Scroll it into your browser viewport
    ctrl.ScrollIntoView()
    sleep(0.3)   # give the page a moment to repaint
    
    # 4) Click it (this also uses the element’s InvokePattern under the hood)
    ctrl.Click()
    print(f"Clicked element #{N} at {ctrl.BoundingRectangle}")

AttributeError: 'TextControl' object has no attribute 'ScrollIntoView'

In [6]:
sleep(2)   # Alt-Tab into your browser

# 1) Grab the Nth TextControl
ctrl = auto.TextControl(
    Name='Nina P.',
    FoundIndex=1
)
if not ctrl.Exists(1, 0.1):
    raise RuntimeError("Element not found in the UIA tree")

# 2) Ask UIA for a clickable point (this will auto-scroll if needed)
x, y, ok = ctrl.GetClickablePoint()
if not ok:
    # if for some reason the pattern isn't supported, fallback to ctrl.Click()
    ctrl.Click()
else:
    # 3) Raw click at that on-screen point
    auto.Click(x, y)

print(f"✅ Clicked at ({x},{y})")

✅ Clicked at (0,0)


### scroll into view

In [41]:
def scroll_into_view(ctrl, max_steps=10, delay=0.2):
    """
    Bring `ctrl` into the visible scroll region:
     1) Use its ScrollItemPattern (if any)
     2) Else page-down inside the closest ScrollPattern ancestor
    Returns True once scrolled.
    """
    # 1) Element-level
    sip = ctrl.GetPattern(PatternId.ScrollItemPattern)
    if sip:
        # IUIAutomationScrollItemPattern::ScrollIntoView
        return sip.ScrollIntoView()                                # :contentReference[oaicite:0]{index=0}

    # 2) Container-level
    ancestor = ctrl.GetAncestorControl(lambda c, d: c.GetPattern(PatternId.ScrollPattern) is not None)
    if not ancestor:
        return False
    scroll = ancestor.GetPattern(PatternId.ScrollPattern)
    for _ in range(max_steps):
        rect = ctrl.BoundingRectangle
        # if the top of `ctrl` is already above the container’s bottom, we’re done
        if rect.top < ancestor.BoundingRectangle.bottom:
            return True
        # IUIAutomationScrollPattern::Scroll(NoAmount, LargeIncrement)
        scroll.Scroll(ScrollAmount.NoAmount, ScrollAmount.LargeIncrement)  # :contentReference[oaicite:1]{index=1}
        sleep(delay)
    return False

### get rectangle bounds as tuple

In [42]:
def get_rect(rect):
    """
    Convert a uiautomation Rect object into a (left, top, right, bottom) tuple.
    Handles different possible attribute names.
    """
    # Try the most common attribute names
    for a in ('left','Left'):
        if hasattr(rect, a):
            left = getattr(rect, a)
            break
    else:
        # maybe it's a .X property plus .Width/.Height
        left = getattr(rect, 'X', None)
    for a in ('top','Top'):
        if hasattr(rect, a):
            top = getattr(rect, a)
            break
    else:
        top = getattr(rect, 'Y', None)
    # Try right/bottom
    if hasattr(rect, 'right'):
        right = rect.right
    elif hasattr(rect, 'Right'):
        right = rect.Right
    elif left is not None and hasattr(rect, 'Width'):
        right = left + rect.Width
    else:
        right = None

    if hasattr(rect, 'bottom'):
        bottom = rect.bottom
    elif hasattr(rect, 'Bottom'):
        bottom = rect.Bottom
    elif top is not None and hasattr(rect, 'Height'):
        bottom = top + rect.Height
    else:
        bottom = None

    if None in (left, top, right, bottom):
        # Last ditch: try unpacking (unlikely to work now)
        try:
            left, top, right, bottom = tuple(rect)
        except Exception:
            raise RuntimeError(f"Can't extract bounds from {rect}")
    return left, top, right, bottom

In [ ]:
sleep(2)   # Alt-Tab into your browser

# 1) Grab the Nth TextControl
ctrl = auto.TextControl(
    Name='✅ Started 2 startups (both fake)',
    FoundIndex=1
)
if not ctrl.Exists(1, 0.1):
    raise RuntimeError("Element not found in the UIA tree")

# 2) Ask UIA for a clickable point (this will auto-scroll if needed)
x, y, ok = ctrl.GetClickablePoint()
if not ok:
    # if for some reason the pattern isn't supported, fallback to ctrl.Click()
    ctrl.Click()
else:
    # 3) Raw click at that on-screen point
    auto.Click(x, y)

print(f"✅ Clicked at ({x},{y})")

In [44]:
sleep(2)
# the_text = "Started 2 startups (both fake)"
the_text = "Nina P."

# 1) Locate the control
ctrl = auto.TextControl(Name=the_text, searchInterval=5, FoundIndex=1)
print(ctrl)
if not ctrl.Exists(1, 0.1):
    raise RuntimeError("Element not found")

# 1) Scroll into view
if not scroll_into_view(ctrl):
    raise RuntimeError("Couldn’t scroll element into view")

# 2) Click in the middle of its bounding box
#    MoveCursorToInnerPos + Click(MouseEvent) under the hood
ctrl.Click()                                               # :contentReference[oaicite:2]{index=2}
print(f"✅ Scrolled to and clicked “{ctrl.Name}”")

2025-06-14 00:47:23.046 2876657370.py[7] <module> -> Find Control Timeout(10s): {FoundIndex: 1, Name: 'Nina P.', ControlType: TextControl}


LookupError: Find Control Timeout(10s): {FoundIndex: 1, Name: 'Nina P.', ControlType: TextControl}

### click or click parent default max_depth  =3

In [46]:
def click_or_parent(ctrl, max_levels=3):
    """
    Scroll `ctrl` into view, try clicking it; if it isn’t clickable,
    climb up to `max_levels` ancestors and click the first one that is.
    """
    # 1) Scroll into view
    if not scroll_into_view(ctrl):
        raise RuntimeError(f"Couldn’t scroll “{ctrl.Name}” into view")

    # 2) Try clicking the control itself
    x, y, ok = ctrl.GetClickablePoint()
    if ok:
        auto.Click(x, y)
        print(f"✅ Clicked target “{ctrl.Name}” at ({x},{y})")
        return

    # 3) Climb up ancestors
    ancestor = ctrl
    for level in range(max_levels):
        ancestor = ancestor.GetParentControl()
        if not ancestor:
            break
        # re-scroll & try its GetClickablePoint
        if scroll_into_view(ancestor):
            x, y, ok = ancestor.GetClickablePoint()
            if ok:
                ancestor.Click()
                print(f"✅ Clicked ancestor level {level+1}: “{ancestor.Name or ancestor.ControlTypeName}”")
                return

    raise RuntimeError(f"No clickable ancestor found within {max_levels} levels")

In [51]:
sleep(2)  # Alt+Tab into LinkedIn

the_text = "✅ Started 2 startups (both fake)"
ctrl = auto.TextControl(Name=the_text, FoundIndex=1)

if not ctrl.Exists(1, 0.1):
    raise RuntimeError(f"Element “{the_text}” not found")

# Instead of ctrl.Click(), call:
click_or_parent(ctrl, max_levels=3)


✅ Clicked ancestor level 3: “Post I just got rejected from Stanford… 😭 But I have 3 nonprofits, 2 startups, and 20,000 followers. That was a DM I got from a high school senior last week. He did everything “right”: ✅ Started 2 startups (both fake) ✅ Launched 3 nonprofits (all dead) ✅ Built a personal brand on LinkedIn ✅ Took 14 APs ✅ Got a 1550 SAT And still... ❌ Rejected. Why? Because colleges - and life - can smell bullsh*t. Admissions officers aren’t stupid. They’ve seen the 500th financial literacy app for teens. They know when a “founder” has no scars. They can smell the polish. And they’re not impressed. The winners? They don't try to impress. They pursue their passions. They fail. They bleed. They talk about their losses. Their doubts. Their fears. What it really cost them. Openly. Honestly. Unapologetically. Because here's the truth: You don't need to be all put together for colleges - or people, in general - to love you. You just have to be open being yourself. And have the gut